In [30]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from utils.feature_selection import *

# Feature Extraction

In [31]:
final_dataset_path = '../dataset/final/windows/'
acc_dataset_path = final_dataset_path + 'PatchTable_Acceleration_Normalised.csv'
gyro_dataset_path = final_dataset_path + 'PatchTable_Gyroscope_Normalised.csv'

In [32]:
acc_dataset_df = pd.read_csv(acc_dataset_path).iloc[:, 1:]
gyro_dataset_df = pd.read_csv(gyro_dataset_path).iloc[:, 1:]
acc_dataset_df = acc_dataset_df.drop('ClassificationType', axis=1)
gyro_dataset_df = gyro_dataset_df.drop('ClassificationType', axis=1)

KeyError: "['ClassificationType'] not found in axis"

### Feature Importance

In [ ]:
# Separate features and target variable
acc_features = acc_dataset_df.drop(columns=['Class', ])
acc_target = acc_dataset_df['Class']
gyro_features = gyro_dataset_df.drop(columns=['Class'])
gyro_target = gyro_dataset_df['Class']

In [ ]:
acc_random_forest_model = RandomForestClassifier()
gyro_random_forest_model = RandomForestClassifier()
acc_random_forest_model.fit(acc_features, acc_target)
gyro_random_forest_model.fit(gyro_features, gyro_target)

In [ ]:
# Get feature importance
acc_feature_importance = acc_random_forest_model.feature_importances_
gyro_feature_importance = gyro_random_forest_model.feature_importances_

acc_best_features = []
gyro_best_features = []

for i in range(len(acc_feature_importance)):
    if acc_feature_importance[i] > 0:
        acc_best_features.append(acc_dataset_df.columns[i + 1])

for i in range(len(gyro_feature_importance)):
    if gyro_feature_importance[i] > 0:
        gyro_best_features.append(gyro_dataset_df.columns[i + 1])


In [ ]:
filtered_acc_dataset_df = pd.concat([acc_dataset_df.iloc[:, :1], acc_dataset_df[acc_best_features].copy()], axis=1)
filtered_gyro_dataset_df = pd.concat([acc_dataset_df.iloc[:, :1], gyro_dataset_df[gyro_best_features].copy()], axis=1)

In [ ]:
filtered_acc_dataset_df

In [ ]:
filtered_gyro_dataset_df

### Correlation Selection

In [ ]:
acc_corr = filtered_acc_dataset_df.corr()
gyro_corr = filtered_gyro_dataset_df.corr()

In [ ]:
acc_corr

In [ ]:
gyro_corr

In [ ]:
acc_feature_corr_matrix = acc_corr.drop(columns=['Class'])
acc_target_corr = acc_feature_corr_matrix.iloc[[0]].copy()
acc_feature_corr_matrix = acc_feature_corr_matrix.drop(acc_feature_corr_matrix.index[0])
gyro_feature_corr_matrix = gyro_corr.drop(columns=['Class'])
gyro_target_corr = gyro_feature_corr_matrix.iloc[[0]].copy()
gyro_feature_corr_matrix = gyro_feature_corr_matrix.drop(gyro_feature_corr_matrix.index[0])

In [ ]:
acc_corr_redundant_features = get_corr_redundant_features(acc_feature_corr_matrix, acc_target_corr)
gyro_corr_redundant_features = get_corr_redundant_features(gyro_feature_corr_matrix, gyro_target_corr)

In [ ]:
print(len(acc_corr_redundant_features))
acc_corr_redundant_features

In [ ]:
print(len(gyro_corr_redundant_features))
gyro_corr_redundant_features

In [ ]:
filtered_acc_dataset_df = filtered_acc_dataset_df.drop(columns=acc_corr_redundant_features)
filtered_gyro_dataset_df = filtered_gyro_dataset_df.drop(columns=gyro_corr_redundant_features)

In [ ]:
filtered_acc_dataset_df

In [ ]:
filtered_gyro_dataset_df

### Saving to CSV

In [ ]:
filtered_acc_dataset_df.to_csv(final_dataset_path + 'PatchTable_Acceleration_Filtered.csv', index=False)
filtered_gyro_dataset_df.to_csv(final_dataset_path + 'PatchTable_Gyroscope_Filtered.csv', index=False)